# **1.** **Frame the Problem and Look at the Big Picture**

# **2.** **Get the Data**

# **3.** **Explore the Data**

# **4.** **Prepare the Data**

# **5.** **Shortlist Promising Models**

# **6.** **Fine-Tune the System**

# **7.** **Present Your Solution**

# **8.** **Launch!**